In [1]:
path = "/Users/amrtamer/Documents/Internship/"

In [2]:
import sys
sys.path.append(path + 'Models')
sys.path.append(path + 'Models/Universal/uni2ts/myenv/lib/python3.10/site-packages/')
import utils as ut
import models

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
# os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

import torch
import matplotlib.pyplot as plt
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from uni2ts.eval_util.plot import plot_single
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import optuna
import torch.nn as nn
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import random
from matplotlib.backends.backend_pdf import PdfPages
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName

In [4]:
data = pd.read_csv(path + r'data/Volve/Volve_cleaned_prepared.csv')
if "Unnamed: 0" in data.columns:
    data = data.drop(columns="Unnamed: 0")
data.head()

,well_name,WELL_BORE_CODE,date,prod_hrs,bhp,bht,dp_tubing,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,whp,...,linear_regressor,EMA_short,EMA_medium,EMA_long,Rolling_short,Rolling_medium,Rolling_long,Lag_short,Lag_medium,Lag_long
0,7405,NO 15/9-F-1 C,2014-04-22,0.268405,1.029311,0.605973,0.396167,-2.016027,-0.533280,3.281836,...,-0.101892,-0.103579,-0.104768,-0.106929,-0.001355,-0.002618,-0.00519,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,7405,NO 15/9-F-1 C,2014-04-23,0.268405,0.845795,0.634777,0.239818,0.088113,-0.419656,2.849981,...,-0.101892,-0.103579,-0.104768,-0.106929,-0.001355,-0.002618,-0.00519,"[0.0, 0.0, -0.10230979661544379]","[0.0, 0.0, 0.0, 0.0, -0.10230979661544379]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,7405,NO 15/9-F-1 C,2014-04-24,0.268405,0.775025,0.639809,0.199890,-0.688434,-0.402874,2.607708,...,-0.101892,-0.009618,-0.053295,-0.079801,-0.001355,-0.002618,-0.00519,"[0.0, -0.10230979661544379, -0.10230979661544379]","[0.0, 0.0, 0.0, -0.10230979661544379, -0.10230...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.10..."
3,7405,NO 15/9-F-1 C,2014-04-25,0.268405,0.705033,0.642084,0.161493,1.145283,-0.379061,2.364038,...,-0.114759,-0.020781,-0.051611,-0.076565,-0.001355,-0.002618,-0.00519,"[-0.10230979661544379, -0.10230979661544379, 0...","[0.0, 0.0, -0.10230979661544379, -0.1023097966...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.1023097..."
4,7405,NO 15/9-F-1 C,2014-04-26,0.268405,0.625357,0.643889,0.117221,1.145283,-0.340115,2.088747,...,0.082297,-0.004428,-0.037199,-0.066767,-0.009496,-0.002618,-0.00519,"[-0.10230979661544379, 0.1777063666263608, -0....","[0.0, -0.10230979661544379, -0.102309796615443...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.102309796615..."


In [5]:
well_info = dict(set([(j, i) for i, j in data[['WELL_BORE_CODE', 'well_name']].values.tolist()])) # creating a dictionary of well identiification with {code: name} structure
well_info

{7078: 'NO 15/9-F-11 H',
 5769: 'NO 15/9-F-5 AH',
 7405: 'NO 15/9-F-1 C',
 5599: 'NO 15/9-F-12 H',
 7289: 'NO 15/9-F-15 D',
 5351: 'NO 15/9-F-14 H'}

In [6]:
univariate_feature = "oil_rate"
days_window = 5
predictions_days = 14
training_gap = days_window + predictions_days

In [8]:
dataset = ut.produce_univariate_dataset(data, days_window, predictions_days, univariate_feature, well_info)
dataset

,well_name,inputs_dates,targets_dates,inputs_index,targets_index,inputs,targets
0,7078,"[2013-07-24, 2013-07-25, 2013-07-26, 2013-07-2...","[2013-07-30, 2013-07-31, 2013-08-03, 2013-08-0...","[428, 429, 430, 431, 432]","[433, 434, 435, 436, 437, 438, 439, 440, 441, ...","[-0.602010272310589, -0.4247721297958977, -0.3...","[-0.2002478875700835, -0.2968328228893544, -0...."
1,7078,"[2013-07-25, 2013-07-26, 2013-07-27, 2013-07-2...","[2013-07-31, 2013-08-03, 2013-08-04, 2013-08-0...","[429, 430, 431, 432, 433]","[434, 435, 436, 437, 438, 439, 440, 441, 442, ...","[-0.4247721297958977, -0.3031676147143909, -0....","[-0.2968328228893544, -0.2681241482667802, -0...."
2,7078,"[2013-07-26, 2013-07-27, 2013-07-29, 2013-07-3...","[2013-08-03, 2013-08-04, 2013-08-05, 2013-08-0...","[430, 431, 432, 433, 434]","[435, 436, 437, 438, 439, 440, 441, 442, 443, ...","[-0.3031676147143909, -0.3068636673712412, -0....","[-0.2681241482667802, -0.1753543155255271, -0...."
3,7078,"[2013-07-27, 2013-07-29, 2013-07-30, 2013-07-3...","[2013-08-04, 2013-08-05, 2013-08-06, 2013-08-0...","[431, 432, 433, 434, 435]","[436, 437, 438, 439, 440, 441, 442, 443, 444, ...","[-0.3068636673712412, -0.41269056636817, -0.20...","[-0.1753543155255271, -0.1803578650708679, -0...."
4,7078,"[2013-07-29, 2013-07-30, 2013-07-31, 2013-08-0...","[2013-08-05, 2013-08-06, 2013-08-07, 2013-08-0...","[432, 433, 434, 435, 436]","[437, 438, 439, 440, 441, 442, 443, 444, 445, ...","[-0.41269056636817, -0.2002478875700835, -0.29...","[-0.1803578650708679, -0.1724398392210148, -0...."
...,...,...,...,...,...,...,...
7874,5351,"[2016-06-21, 2016-06-22, 2016-06-23, 2016-06-2...","[2016-06-26, 2016-06-27, 2016-06-28, 2016-06-2...","[7077, 7078, 7079, 7080, 7081]","[7082, 7083, 7084, 7085, 7086, 7087, 7088, 708...","[-0.8787876415334117, -0.878955643926905, -0.8...","[-0.8792186041949813, -0.8802339230078314, -0...."
7875,5351,"[2016-06-22, 2016-06-23, 2016-06-24, 2016-06-2...","[2016-06-27, 2016-06-28, 2016-06-29, 2016-06-3...","[7078, 7079, 7080, 7081, 7082]","[7083, 7084, 7085, 7086, 7087, 7088, 7089, 709...","[-0.878955643926905, -0.878911817215559, -0.87...","[-0.8802339230078314, -0.8796495668565508, -0...."
7876,5351,"[2016-06-23, 2016-06-24, 2016-06-25, 2016-06-2...","[2016-06-28, 2016-06-29, 2016-06-30, 2016-07-0...","[7079, 7080, 7081, 7082, 7083]","[7084, 7085, 7086, 7087, 7088, 7089, 7090, 709...","[-0.878911817215559, -0.8788387726966489, -0.8...","[-0.8796495668565508, -0.8795619134338586, -0...."
7877,5351,"[2016-06-24, 2016-06-25, 2016-06-26, 2016-06-2...","[2016-06-29, 2016-06-30, 2016-07-01, 2016-07-0...","[7080, 7081, 7082, 7083, 7084]","[7085, 7086, 7087, 7088, 7089, 7090, 7091, 709...","[-0.8788387726966489, -0.8794231288479296, -0....","[-0.8795619134338586, -0.8802266185559404, -0...."


In [7]:
# dataset.to_pickle(path + "/data/Volve/all_data_univariate.pkl")
# dataset

In [15]:
train_size = 0.6
val_size = 0.2
test_size = 0.2

In [16]:
test_size_param1 = test_size
test_size_param2 = val_size / (1 - test_size_param1)

In [ ]:
data_train, data_val, data_test = ut.split_dataset(dataset, val_size, test_size, well_info, shuffle_train=True)

In [10]:


# data_train.to_pickle(path + "/data/Volve/data_univariate_train.pkl")
# data_val.to_pickle(path + "/data/Volve/data_univariate_val.pkl")
# data_test.to_pickle(path + "/data/Volve/data_univariate_test.pkl")

In [10]:
all_data = pd.read_pickle(path + "data/Volve/all_data_univariate.pkl")
data_train = pd.read_pickle(path + "data/Volve/data_univariate_train.pkl")
data_val = pd.read_pickle(path + "data/Volve/data_univariate_val.pkl")
data_test = pd.read_pickle(path + "data/Volve/data_univariate_test.pkl")
data_train.shape, data_val.shape, data_test.shape

((4730, 7), (1577, 7), (1573, 7))

In [11]:
batch_size = 50

In [13]:
train_dataset = ut.UnivariateTimeSeriesDataset(data_train)
val_dataset = ut.UnivariateTimeSeriesDataset(data_val)
test_dataset = ut.UnivariateTimeSeriesDataset(data_test)


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [14]:
visualization_test_dataset = ut.UnivariateVisualizationTimeSeriesDataset(data_test)

visualization_test_loader = DataLoader(visualization_test_dataset, batch_size=int(len(test_dataset) / 50), shuffle=True)


In [12]:
data_test

,well_name,inputs_dates,targets_dates,inputs_index,targets_index,inputs,targets
0,7405,"[2015-09-19, 2015-09-20, 2015-10-14, 2015-10-1...","[2015-10-17, 2015-10-18, 2015-10-19, 2015-10-2...","[328, 329, 330, 331, 332]","[333, 334, 335, 336, 337, 338, 339, 340, 341, ...","[-0.5821100235282172, -0.72420964887026, -0.75...","[-0.8418928550319287, -0.8446247200391658, -0...."
1,7405,"[2015-09-20, 2015-10-14, 2015-10-15, 2015-10-1...","[2015-10-18, 2015-10-19, 2015-10-20, 2015-10-2...","[329, 330, 331, 332, 333]","[334, 335, 336, 337, 338, 339, 340, 341, 342, ...","[-0.72420964887026, -0.7578913833006464, -0.87...","[-0.8446247200391658, -0.7896733284631101, -0...."
2,7405,"[2015-10-14, 2015-10-15, 2015-10-16, 2015-10-1...","[2015-10-19, 2015-10-20, 2015-10-21, 2015-10-2...","[330, 331, 332, 333, 334]","[335, 336, 337, 338, 339, 340, 341, 342, 343, ...","[-0.7578913833006464, -0.8771952710211721, -0....","[-0.7896733284631101, -0.7931575520151211, -0...."
3,7405,"[2015-10-15, 2015-10-16, 2015-10-17, 2015-10-1...","[2015-10-20, 2015-10-21, 2015-10-22, 2015-10-2...","[331, 332, 333, 334, 335]","[336, 337, 338, 339, 340, 341, 342, 343, 344, ...","[-0.8771952710211721, -0.8390514232463265, -0....","[-0.7931575520151211, -0.8022515946194264, -0...."
4,7405,"[2015-10-16, 2015-10-17, 2015-10-18, 2015-10-1...","[2015-10-21, 2015-10-22, 2015-10-23, 2015-10-2...","[332, 333, 334, 335, 336]","[337, 338, 339, 340, 341, 342, 343, 344, 345, ...","[-0.8390514232463265, -0.8418928550319287, -0....","[-0.8022515946194264, -0.7951881896408215, -0...."
...,...,...,...,...,...,...,...
1568,5769,"[2016-08-04, 2016-08-05, 2016-08-06, 2016-08-0...","[2016-08-09, 2016-08-10, 2016-08-11, 2016-08-1...","[7964, 7965, 7966, 7967, 7968]","[7969, 7970, 7971, 7972, 7973, 7974, 7975, 797...","[-0.7225746333923075, -0.7166507229086997, -0....","[-0.7318767962210462, -0.7237141278873048, -0...."
1569,5769,"[2016-08-05, 2016-08-06, 2016-08-07, 2016-08-0...","[2016-08-10, 2016-08-11, 2016-08-12, 2016-08-1...","[7965, 7966, 7967, 7968, 7969]","[7970, 7971, 7972, 7973, 7974, 7975, 7976, 797...","[-0.7166507229086997, -0.7226768957187816, -0....","[-0.7237141278873048, -0.7192072810705527, -0...."
1570,5769,"[2016-08-06, 2016-08-07, 2016-08-08, 2016-08-0...","[2016-08-11, 2016-08-12, 2016-08-13, 2016-08-1...","[7966, 7967, 7968, 7969, 7970]","[7971, 7972, 7973, 7974, 7975, 7976, 7977, 797...","[-0.7226768957187816, -0.7202079909796208, -0....","[-0.7192072810705527, -0.7133929373653101, -0...."
1571,5769,"[2016-08-07, 2016-08-08, 2016-08-09, 2016-08-1...","[2016-08-12, 2016-08-13, 2016-08-14, 2016-08-1...","[7967, 7968, 7969, 7970, 7971]","[7972, 7973, 7974, 7975, 7976, 7977, 7978, 797...","[-0.7202079909796208, -0.7207996515827926, -0....","[-0.7133929373653101, -0.7128670168291575, -0...."


In [15]:
size = "small"
num_samples = 50

In [16]:
# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters
    num_samples = trial.suggest_int("num_samples", 10, 50, step=10)
    size = trial.suggest_categorical("size", ["tiny", "mini", "small", "base", "large"])
    # Load the model
    pipeline = ChronosPipeline.from_pretrained(
        f"amazon/chronos-t5-{size}",
        device_map="mps",  # use "cpu" for CPU inference and "mps" for Apple Silicon
        torch_dtype=torch.bfloat16,  # torch.bfloat16,
    )
    
    avg_loss = 0
    
    for X_batch, Y_batch in val_loader:
        
        forecast = pipeline.predict(
            context=X_batch,
            prediction_length=predictions_days,
            num_samples=num_samples,
        )
        
        prediction = forecast.mean(axis=1)
        prediction = prediction.view(prediction.shape[0] * prediction.shape[1])
        Y_batch = Y_batch.view(Y_batch.shape[0] * Y_batch.shape[1])
        
        loss = F.mse_loss(prediction, Y_batch)
        avg_loss += loss.item()
    
    val_loss = avg_loss / len(val_loader)
    return val_loss

In [14]:
# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# Get the best hyperparameters
best_params = study.best_params
print(f"Best parameters: {best_params}")

[I 2024-07-08 14:02:15,492] A new study created in memory with name: no-name-561035a0-8901-401f-926f-1e5fa9fdb5e0
/Users/amrtamer/Library/Python/3.9/lib/python/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
[I 2024-07-08 14:03:14,934] Trial 0 finished with value: 0.07399260968668386 and parameters: {'num_samples': 50, 'size': 'small'}. Best is trial 0 with value: 0.07399260968668386.
[I 2024-07-08 14:03:50,849] Trial 1 finished with value: 0.08172283679596148 and parameters: {'num_samples': 20, 'size': 'small'}. Best is trial 0 with value: 0.07399260968668386.
[I 2024-07-08 14:04:20,112] Trial 2 finished with value: 0.06905774885672145 and parameters: {'num_samples': 30, 'size': 'mini'}. Best is trial 2 with value: 0.06905774885672145.
[I 2024-07-08 14:09:04,553] Trial 3 finishe

Best parameters: {'num_samples': 50, 'size': 'tiny'}


In [27]:
size = "small"
num_samples = 50

In [28]:
model = models.Moirai(size=size, input_length=days_window, output_length=predictions_days, num_samples=num_samples)

In [19]:
train_metrics = ut.evaluate(model, train_loader)
val_metrics = ut.evaluate(model, val_loader)
test_metrics = ut.evaluate(model, test_loader)

Metrics on dataset:

Loss is 68234351.7811
MAE is 125.3311
RMSE is 8273.4922
R² is -63142952.0000


Metrics on dataset:

Loss is 4996661576.4665
MAE is 736.6165
RMSE is 67147.2969
R² is -4066785280.0000


Metrics on dataset:

Loss is 0.0026
MAE is 0.0335
RMSE is 0.0509
R² is 0.8916




In [18]:
ut.produce_graphs(model, visualization_test_loader, data, data_test, days_window, predictions_days, path + "Models/Universal/Moirai/Graphs", well_info, test_size)
test_metrics = ut.evaluate(model, test_loader)

Metrics on dataset:

Loss is 0.0026
MAE is 0.0334
RMSE is 0.0509
R² is 0.8918




In [51]:
dataset_history = ut.produce_univariate_dataset(data, 1, predictions_days, univariate_feature, well_info)
_, _, data_test_history = ut.split_dataset(dataset_history, val_size, test_size, well_info, shuffle_train=True)
test_dataset_history = ut.UnivariateTimeSeriesHistoryDataset(data_test_history, data)
test_loader_history = DataLoader(test_dataset_history, batch_size=1, shuffle=False)

In [56]:
class Moirai_history(nn.Module):
    def __init__(self, size, output_length, num_samples):
        super().__init__()
        self.size = size
        self.output_length = output_length
        self.num_samples = num_samples
        
    
    def forward(self, x, targets=None):
        B, T = x.shape
        self.model = MoiraiForecast(
            module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.0-R-{self.size}"),
            prediction_length=self.output_length,
            context_length=T,
            patch_size="auto", # patch size: choose from {"auto", 8, 16, 32, 64, 128}
            num_samples=self.num_samples,
            target_dim=1,
            feat_dynamic_real_dim=0, # ds.num_feat_dynamic_real,
            past_feat_dynamic_real_dim=0 # ds.num_past_feat_dynamic_real,
        )
        predictor = self.model.create_predictor(batch_size=B)
        time_series_list = [{
            FieldName.TARGET: inputs.cpu().numpy().flatten().tolist(),
            FieldName.START: pd.Timestamp("2020-01-01")  # Same start date for all series; adjust if needed
        } for inputs in x]
        
        dataset_x = ListDataset(
            time_series_list,
            freq="1D"
        )
        
        forecasts = predictor.predict(dataset_x)
        outputs = torch.tensor(np.array([forecast.samples.mean(axis=0) for forecast in forecasts]))
        if targets is None:
            loss = None 
        else:
            logits = outputs.view(-1)
            targets = targets.view(-1)  # Ensure targets have the same shape as outputs
            loss = F.mse_loss(logits, targets)
        
        return outputs, loss

In [64]:
model = Moirai_history(size=size, output_length=predictions_days, num_samples=num_samples)

In [60]:
ut.produce_graphs(model, visualization_test_loader, data, data_test, days_window, predictions_days, path + "Models/Universal/Moirai/Graphs/History", well_info, test_size)
test_metrics = ut.evaluate(model, test_loader_history)

Metrics on dataset:

Loss is 0.0257
MAE is 0.0742
RMSE is 0.1603
R² is -0.0648


